In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
Data = pd.read_csv("train_black_friday.csv")
Data = Data.drop(columns=["User_ID", "Product_ID"])

X = Data.drop("Purchase", axis=1)
Y = Data["Purchase"].values

In [3]:
for col in X.select_dtypes(exclude=["object"]).columns:
    X[col] = X[col].fillna(X[col].median())


for col in X.select_dtypes(include=["object"]).columns: #CodeGeeks snippet
    X[col] = X[col].fillna(X[col].mode()[0])
    X[col] = X[col].astype('category').cat.codes

In [4]:
scaler = StandardScaler() #CodeGeeks snippet
num_cols = X.select_dtypes(include=[np.number]).columns
X[num_cols] = scaler.fit_transform(X[num_cols])

X_processed = X.values
X_train, X_test, y_train, y_test = train_test_split(X_processed, Y, test_size=0.2, random_state=42)

In [21]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

model.compile(optimizer=Adam(0.001), metrics=['mean_squared_error'], loss='mean_squared_error')

In [22]:
# Train
history = model.fit(
    X_train,
    y_train,
    epochs=8,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/8
11002/11002 ━━━━━━━━━━━━━━━━━━━━ 29s 3ms/step - loss: 21452450.0000 - mean_squared_error: 21452450.0000 - val_loss: 17693342.0000 - val_mean_squared_error: 17693342.0000
Epoch 2/8
11002/11002 ━━━━━━━━━━━━━━━━━━━━ 25s 2ms/step - loss: 17945670.0000 - mean_squared_error: 17945670.0000 - val_loss: 15550518.0000 - val_mean_squared_error: 15550518.0000
Epoch 3/8
11002/11002 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - loss: 16263426.0000 - mean_squared_error: 16263426.0000 - val_loss: 14382757.0000 - val_mean_squared_error: 14382757.0000
Epoch 4/8
11002/11002 ━━━━━━━━━━━━━━━━━━━━ 25s 2ms/step - loss: 15922461.0000 - mean_squared_error: 15922461.0000 - val_loss: 14333723.0000 - val_mean_squared_error: 14333723.0000
Epoch 5/8
11002/11002 ━━━━━━━━━━━━━━━━━━━━ 25s 2ms/step - loss: 15823203.0000 - mean_squared_error: 15823203.0000 - val_loss: 14374922.0000 - val_mean_squared_error: 14374922.0000
Epoch 6/8
11002/11002 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - loss: 15795910.0000 - mean_squared_error: 

In [ ]:
preds = model.predict(X_test)
mae = mean_absolute_error(y_test, preds)
print("Check 4 RMSE:", round(mae, 2))

3438/3438 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step
Check 4 RMSE: 3068.33


In [11]:
y_train.ravel()

array([7800, 8677, 7966, ..., 6954, 6151, 7146], shape=(440054,))

In [18]:
from sklearn.ensemble import RandomForestRegressor as rfr

rfr_mdl = rfr(n_estimators=100)
rfr_mdl.fit(X_train, y_train)

rfr_pred = rfr_mdl.predict(X_test)
print(rfr_pred)

rfr_mae = mean_absolute_error(y_test, rfr_pred)
print("Check 4 RMSE:", round(rmse, 2))

[13029.36780952 13815.64430594  6867.38815837 ...  6521.52992489
  3790.83492424  9501.66880493]
Check 4 RMSE: 3068.33


In [ ]:
preds = model.predict(X_test)